In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path
from compile_sweep_results import compile_sweep_results

In [ ]:
root = Path("/media/nick/hdd021/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/symmetry_breaking/pde/sweeps/")
sweep_name = "sweep01"
df = compile_sweep_results(output_dir=root / sweep_name)
print(df.shape)
df.head()

In [ ]:
fig = px.scatter(df, x="roi_frac", y="max_fold_change", log_y=True)
# add a line at y=1
fig.add_hline(y=1, line_dash="dash", line_color="white", annotation_text="y=1", annotation_position="top left")
# add vertical line at x=500/3500
fig.add_vline(x=500/3500, line_dash="dash", line_color="white", annotation_text="x=500/3500", annotation_position="top right")
# set axis limits
fig.update_xaxes(title_text="Fraction of ROI", range=[0, 1])
fig.update_yaxes(title_text="Max Fold Change", range=[-4, 4])

fig.show()

In [ ]:
df.shape